In [3]:
import pandas as pd
from tqdm import tqdm
import random

In [81]:
df = pd.read_csv('../bkk-budget-with-entry.csv',low_memory=False)

In [82]:
for _, entry_df in df[~df.entry_label.isna()].groupby(['entry_label']):
    entry_df = entry_df.sort_values(['line_label', 'y0'])

In [83]:
print('total lines',len(df.groupby(['pdf', 'pagenum', 'line_label'])))

total lines 449373


In [85]:
pdf_pagenum = list(df.groupby(['pdf', 'pagenum']))
len(pdf_pagenum)

17593

In [86]:
len([p[1].index for p in pdf_pagenum if (~p[1].entry_label.isna()).any() and len(p[1].line_label.unique()) < 100]),\
len([p[1].index for p in pdf_pagenum if ( p[1].entry_label.isna()).any() and len(p[1].line_label.unique()) < 100])

(1039, 17466)

In [87]:
seleted_page_index = random.choices(
    [p[1].index for p in pdf_pagenum if (~p[1].entry_label.isna()).any() and len(p[1].line_label.unique()) < 200]
    , k=500) \
+ random.choices(
    [p[1].index for p in pdf_pagenum if (p[1].entry_label.isna()).any() and len(p[1].line_label.unique()) < 200]
    , k=130)

In [88]:
len(seleted_page_index)

630

In [89]:
indexes = []
for page_index in seleted_page_index:
    indexes += list(page_index)

In [90]:
line_groupby = list(df.loc[indexes].groupby(['pdf', 'pagenum', 'line_label']))
# line_groupby = list(df.groupby(['pdf', 'pagenum', 'line_label']))

In [91]:
len(df.loc[indexes].groupby(['pdf', 'pagenum',]))

524

In [92]:
text_lines = []
prev_entry_label = None
for (pdf_name, page, line_num), line_df in tqdm(line_groupby):
    line_df     = line_df.sort_values(['line_label', 'x0'])
    text_line   = ' '.join(line_df.fix_text.astype(str))
    is_entry    = -1 if line_df.entry_label.isna().all()  else line_df.entry_label.iloc[0]
    entry_label = -1 if line_df.bullet_label.isna().all() else line_df.bullet_label.iloc[0]
    text_lines.append({
        'text': text_line,
        'is_entry': is_entry,
        'entry_label': entry_label,
        'page': page,
        'line_num': line_num,
        'pdf_name': pdf_name
    })

100%|████████████████████████████████████| 12820/12820 [00:17<00:00, 743.46it/s]


In [93]:
line_df = pd.DataFrame(text_lines)

In [94]:
line_df.groupby('pdf_name').text.count().mean()

48.745247148288975

In [95]:
print('ratio (entry line / all): ',(line_df.is_entry != -1).sum()/len(line_df.is_entry))

ratio (entry line / all):  0.6064742589703588


In [99]:
line_df['tag'] = 'O'
for x, entry_df in line_df[line_df.is_entry != -1].groupby(['is_entry']):
    line_df.loc[entry_df.iloc[0].name, 'tag'] = 'B-ENTRY'
    line_df.loc[entry_df.iloc[1:].index, 'tag'] = 'I-ENTRY'

In [98]:
i = random.randint(0,1000)
line_df[i:i+10]

,text,is_entry,entry_label,page,line_num,pdf_name,tag,end
233,และบริหารจัดการโรงควบคุมคุณภาพน้ำจตุจักร และบร...,214.0,-1.0,16,17.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
234,"ระยะที่ ระยะที่ 3 3 110,800,000 110,800,000 บา...",214.0,-1.0,16,18.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
235,7. 7. โครงการจ้างที่ปรึกษาสำรวจ โครงการจ้างที่...,215.0,4.0,16,19.0,pdf/61/A20171003161852.pdf,B-ENTRY,NaN
236,ระบบรวบรวมน้ำเสีย ระบบรวบรวมน้ำเสีย (เพิ่มเติม...,215.0,-1.0,16,20.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
237,เข้าโรงควบคุมคุณภาพน้ำดินแดง เข้าโรงควบคุมคุณภ...,215.0,-1.0,16,21.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
238,8. 8. โครงการก่อสร้างระบบรวบรวมน้ำเสีย โครงการ...,216.0,4.0,16,22.0,pdf/61/A20171003161852.pdf,B-ENTRY,NaN
239,จากพื้นที่รับน้ำคลองลาดโตนด จากพื้นที่รับน้ำคล...,216.0,-1.0,16,23.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
240,9. 9. โครงการปรับปรุงโรงควบคุมคุณภาพน้ำ โครงกา...,217.0,4.0,16,24.0,pdf/61/A20171003161852.pdf,B-ENTRY,NaN
241,"ห้วยขวาง ห้วยขวาง 4,800,000 4,800,000 บาท บาท",217.0,-1.0,16,25.0,pdf/61/A20171003161852.pdf,O,I-ENTRY
242,10. 10. โครงการก่อสร้างระบบรวบรวมน้ำเสีย โครงก...,218.0,4.0,16,26.0,pdf/61/A20171003161852.pdf,B-ENTRY,NaN


In [100]:
line_df.to_csv('../dataset/bullet_train-set.csv')

# Rearrange Columns

In [335]:
df = pd.read_csv('../bkkbudget_61-64_v0.1.csv', index_col=0)
df = df.assign(fisical_year=df.pdf_name.apply(lambda pdf_name: int(pdf_name.split('/')[1])))

In [336]:
sub_cat_cols = [col for col in df.columns if col[-1].isdigit()]
cols = [
    'fisical_year','name_organization', 'pdf_name', *sub_cat_cols, 
    'output/proj', 'output_proj_name', 'amount', 'pdf_link']
df = df[cols]

In [337]:
df.to_csv('../bkkbudget_61-64_v0.2.csv')